In [2]:
import json
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re
from requests.compat import urlparse, urljoin
import sqlite3
from requests import Session
from datetime import datetime, timedelta 

In [6]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params = params if method.upper() == 'GET' else '',
                               data = params if method.upper() == 'POST' else '',
                               headers = headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60) # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [ ]:
conn = sqlite3.connect('news.db')
cur = conn.cursor()
cur.executescript('''
    DROP TABLE IF EXISTS head;
    CREATE TABLE head(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        head TEXT NOT NULL,
        cdate TEXT NOT NULL,
        wdate TEXT NOT NULL,
        ref INTEGER NOT NULL,
        page INTEGER NOT NULL
    );
''')

cur.executescript('''
    DROP TABLE IF EXISTS history;
    CREATE TABLE history(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        seen TEXT NOT NULL,
        ref INTEGER NOT NULL
    );
''')

In [ ]:
seen = list()
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
          'cookie': '_ga=GA1.2.155331207.1590371425; _ss_pp_id=6d5c749e45df5fc2e3f1588636203804; __gads=ID=32309e63460e7bec:T=1590587589:S=ALNI_MbT83O-w-NBELV4M2sBrRwfoU4ZtQ; PCID=15946360484484587904430; OAX=gIYFPF8MNw4AAKeC; __utmz=222464713.1594814175.4.2.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); adfit_sdk_id=27c83848-9b3d-4f52-bd4d-abbd9d93a0ef; __utma=222464713.155331207.1590371425.1594814175.1595161152.5; _td=8dc8df98-07d4-4639-aa8b-f5bcd75c27b4; _cb_ls=1; _cb=BwJni3Czf8gSDbs39n; _gid=GA1.2.780937476.1597652887; _chartbeat2=.1597235265832.1597657270138.100001.P_loMBN6GXFO-uI2DZimj6lyt7i.1; _gat=1; _gat_chosun_total=1',
          'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6'}

# 시간변수 만들기
start_date = datetime.strptime('20190817', '%Y%m%d') 
end_date = datetime.today()

str_date_list = [] 
while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): 
    str_date_list.append(start_date.strftime('%Y%m%d')) 
    start_date += timedelta(days=1)

url = 'https://news.chosun.com/svc/list_in/list_title.html?indate='
urls = list()
for i in str_date_list:
    u = url + i + '&source=1&pn=1'
    urls.append(u)    

count = 0
    
while urls:
    try:
        count += 1
        seed = urls.pop(-1)

        resp = download(seed, headers=headers)
        dom = BeautifulSoup(resp.content.decode('utf-8','replace'), 'html.parser')
        seen.append(seed)
        cur.execute('''
            INSERT INTO history(seen, ref) VALUES(?, 1000)
        ''', [seed])
        
        for _ in [_['href'] for _ in dom.select('#list_body_id > div.paginate > ul > li > a') 
                  if _.has_attr('href') or _.has_attr('next')]:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen:
                urls.append(newUrls)
        
        if dom.select('#list_body_id > div.list_content > dl > dt') != None:
            head = [_.text.strip() for _ in dom.select('#list_body_id > div.list_content > dl > dt') ]
            wdate = [_.text.strip() for _ in dom.select('#list_body_id > div.list_content > dl > dd.date_author > span.date') ]
            cdate = str(datetime.now()).split('.')[0]
            page = re.search('pn=(\d+)', urlparse(seed).query).group(1)
            if len(head)==len(wdate):
                for _ in range(0,len(head)):
                    cur.execute('''
                        INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,1000)
                        ''', [head[_], wdate[_], cdate, page])
                    conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)